## Reference Image

In [12]:
# Load your images
img1 = cv2.imread('D:/SDP/Imfusion-main/screenshots/medical1.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('D:/SDP/Imfusion-main/screenshots/medical2.png', cv2.IMREAD_GRAYSCALE)
fused_img = cv2.imread('D:/SDP/Imfusion-main/Fused_Images/fused.png', cv2.IMREAD_GRAYSCALE)

def resize_images(*images):
    # Find the smallest dimensions in the set
    min_height = min(image.shape[0] for image in images)
    min_width = min(image.shape[1] for image in images)
    resized_images = [cv2.resize(img, (min_width, min_height), interpolation=cv2.INTER_AREA) for img in images]
    return resized_images


img1, img2, fused_img = resize_images(img1, img2, fused_img)

In [13]:
import numpy as np

def compute_ncc(image1, image2):
    """
    Compute the Nonlinear Correlation Coefficient (NCC) between two images.
    
    Parameters:
        image1: numpy.ndarray, source image 1
        image2: numpy.ndarray, source image 2
    
    Returns:
        float, NCC value
    """
    mean1 = np.mean(image1)
    mean2 = np.mean(image2)
    std1 = np.std(image1)
    std2 = np.std(image2)
    
    NCC = np.mean((image1 - mean1) * (image2 - mean2)) / (std1 * std2)
    
    return NCC

def compute_ncc_matrix(img1, img2, fused_img):
    """
    Compute the nonlinear correlation matrix R using NCC values.
    
    Parameters:
        img1: numpy.ndarray, source image 1
        img2: numpy.ndarray, source image 2
        fused_img: numpy.ndarray, fused image
    
    Returns:
        numpy.ndarray, nonlinear correlation matrix R
    """
    NCCA_B = compute_ncc(img1, img2)
    NCCA_F = compute_ncc(img1, fused_img)
    NCCB_A = compute_ncc(img2, img1)
    NCCB_F = compute_ncc(img2, fused_img)
    NCCF_A = compute_ncc(fused_img, img1)
    NCCF_B = compute_ncc(fused_img, img2)
    
    R = np.array([[1, NCCA_B, NCCA_F],
                  [NCCB_A, 1, NCCB_F],
                  [NCCF_A, NCCF_B, 1]])
    
    return R

def compute_qncie(img1, img2, fused_img):
    """
    Compute the Nonlinear Correlation Information Entropy (QNCIE) between source images and fused image.
    
    Parameters:
        img1: numpy.ndarray, source image 1
        img2: numpy.ndarray, source image 2
        fused_img: numpy.ndarray, fused image
    
    Returns:
        float, QNCIE value
    """
    R = compute_ncc_matrix(img1, img2, fused_img)
    eigenvalues = np.linalg.eigvals(R)
    qncie = 1 + np.sum((eigenvalues/3) * np.log2(eigenvalues/3)) / np.log2(256)
    
    return qncie

# Example usage:
qncie_value = compute_qncie(img1, img2, fused_img)
print("QNCIE value:", qncie_value)


QNCIE value: 0.922940599642658


The QNCIE value of 0.9229 suggests a high degree of nonlinear correlation between the fused image and the source images. In other words, the fusion process has effectively preserved the nonlinear relationships present in the original images.

In terms of image fusion quality, this high QNCIE value indicates that the fused image captures the essential features and characteristics of both source images while maintaining strong correlations with them. Therefore, the overall impression is that the fusion image quality is excellent, as it effectively combines information from the source images while preserving their nonlinear relationships.

    Low QNCIE (0 to 0.3):
        Indicates poor fusion quality.
        The fused image may not effectively capture the essential features of the source images.
        There is minimal correlation between the fused image and the source images, suggesting a lack of information preservation.

    Moderate QNCIE (0.3 to 0.7):
        Suggests moderate fusion quality.
        The fused image retains some features from the source images, but there may be some loss of detail or fidelity.
        Correlation between the fused image and the source images is moderate, indicating partial preservation of information.

    High QNCIE (0.7 to 1):
        Indicates excellent fusion quality.
        The fused image effectively combines information from the source images while preserving their essential features and characteristics.
        There is strong correlation between the fused image and the source images, suggesting a high degree of information preservation.



In [14]:
from sklearn.feature_selection import mutual_info_regression

def compute_mi(image1, image2):
    """
    Compute Mutual Information (MI) between two images.
    
    Parameters:
        image1: numpy.ndarray, source image 1
        image2: numpy.ndarray, source image 2
    
    Returns:
        float, MI value
    """
    # Flatten images to 1D arrays
    flat_image1 = image1.flatten()
    flat_image2 = image2.flatten()
    
    # Compute mutual information
    mi = mutual_info_regression(flat_image1.reshape(-1, 1), flat_image2)[0]
    
    return mi

def compute_fmi(img1, img2, fused_img):
    """
    Compute the Feature Mutual Information (FMI) between source images and fused image.
    
    Parameters:
        img1: numpy.ndarray, source image 1
        img2: numpy.ndarray, source image 2
        fused_img: numpy.ndarray, fused image
    
    Returns:
        float, FMI value
    """
    # Compute Mutual Information between each source image and fused image
    mi_af = compute_mi(img1, fused_img)
    mi_bf = compute_mi(img2, fused_img)
    
    # Sum up the Mutual Information values
    fmi = mi_af + mi_bf
    
    return fmi

# Example usage:
fmi_value = compute_fmi(img1, img2, fused_img)
print("FMI value:", fmi_value)


FMI value: 3.0720635497365



    Low FMI (0 to 1):
        Indicates poor fusion quality.
        The fused image may lack important characteristic information from the source images.
        There is minimal transmission of features to the fused image, resulting in a loss of distinctive characteristics.

    Moderate FMI (1 to 2):
        Suggests moderate fusion quality.
        The fused image retains some characteristic information from the source images, but there may be some loss of detail or clarity.
        Features from the source images are partially transmitted to the fused image, resulting in a moderate fusion effect.

    High FMI (2 and above):
        Indicates excellent fusion quality.
        The fused image effectively combines characteristic information from the source images.
        There is a significant transmission of features to the fused image, resulting in a high-quality fusion effect.

In summary, a higher FMI value indicates better fusion quality, as it signifies a greater transmission of characteristic information from the source images to the fused image. In this case, the FMI value of 3.072 suggests a high-quality fusion with a significant transmission of features, resulting in an effective combination of information from the source images.

In [15]:
import numpy as np

def compute_entropy(image):
    """
    Compute the entropy of an image.
    
    Parameters:
        image: numpy.ndarray, input image
    
    Returns:
        float, entropy value
    """
    # Flatten image to 1D array
    flat_image = image.flatten()
    
    # Compute histogram
    hist, _ = np.histogram(flat_image, bins=256, density=True)
    
    # Remove zeros to avoid log(0) issues
    hist = hist[hist != 0]
    
    # Compute entropy
    entropy = -np.sum(hist * np.log2(hist))
    
    return entropy

def compute_mi(image1, image2):
    """
    Compute Mutual Information (MI) between two images.
    
    Parameters:
        image1: numpy.ndarray, source image 1
        image2: numpy.ndarray, source image 2
    
    Returns:
        float, MI value
    """
    # Flatten images to 1D arrays
    flat_image1 = image1.flatten()
    flat_image2 = image2.flatten()
    
    # Compute joint histogram
    joint_hist, _, _ = np.histogram2d(flat_image1, flat_image2, bins=256, density=True)
    
    # Remove zeros to avoid log(0) issues
    joint_hist = joint_hist[joint_hist != 0]
    
    # Compute mutual information
    mi = -np.sum(joint_hist * np.log2(joint_hist))
    
    return mi

def compute_nmi(img1, img2, fused_img):
    """
    Compute the Normalized Mutual Information (NMI) between source images and fused image.
    
    Parameters:
        img1: numpy.ndarray, source image 1
        img2: numpy.ndarray, source image 2
        fused_img: numpy.ndarray, fused image
    
    Returns:
        float, NMI value
    """
    # Compute entropy of each image
    entropy_a = compute_entropy(img1)
    entropy_b = compute_entropy(img2)
    entropy_f = compute_entropy(fused_img)
    
    # Compute mutual information between each source image and fused image
    mi_af = compute_mi(img1, fused_img)
    mi_bf = compute_mi(img2, fused_img)
    
    # Compute NMI
    nmi = (2 * (mi_af / (entropy_a + entropy_f))) + (2 * (mi_bf / (entropy_b + entropy_f)))
    
    return nmi

# Example usage:
nmi_value = compute_nmi(img1, img2, fused_img)
print("NMI value:", nmi_value)


NMI value: 3.0368034191887237


The Normalized Mutual Information (NMI) value ranges and their corresponding interpretations are as follows:

    Low NMI (0 to 1):
        Indicates poor fusion quality.
        The fused image may not effectively capture the essential features or characteristics of the source images.
        There is minimal mutual information shared between the source images and the fused image.

    Moderate NMI (1 to 2):
        Suggests moderate fusion quality.
        The fused image retains some mutual information from the source images, but there may be some loss of detail or fidelity.
        Mutual information shared between the source images and the fused image is moderate, indicating partial preservation of information.

    High NMI (2 and above):
        Indicates excellent fusion quality.
        The fused image effectively combines mutual information from the source images.
        There is a significant amount of mutual information shared between the source images and the fused image, resulting in a high-quality fusion effect.

In this case, the NMI value of 3.0368 suggests a high-quality fusion with a significant amount of mutual information shared between the source images and the fused image. Therefore, the overall impression is that the fusion quality is excellent, as the fused image effectively combines information from the source images while preserving their essential characteristics.

In [2]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error

def mutual_information(image1, image2):
    hist_2d, _, _ = np.histogram2d(image1.ravel(), image2.ravel(), bins=256)
    pxy = hist_2d / float(np.sum(hist_2d))
    px = np.sum(pxy, axis=1)
    py = np.sum(pxy, axis=0)
    px_py = px[:, None] * py[None, :]
    nzs = pxy > 0
    return np.sum(pxy[nzs] * np.log(pxy[nzs] / px_py[nzs]))

def resize_images(*images):
    # Find the smallest dimensions in the set
    min_height = min(image.shape[0] for image in images)
    min_width = min(image.shape[1] for image in images)
    resized_images = [cv2.resize(img, (min_width, min_height), interpolation=cv2.INTER_AREA) for img in images]
    return resized_images

def calculate_metrics(img1, img2, fused_img):
    metrics = {}
    # Resize images to the smallest common dimensions
    img1, img2, fused_img = resize_images(img1, img2, fused_img)
    metrics['SSIM_CT'] = ssim(img1, fused_img, data_range=fused_img.max() - fused_img.min())
    metrics['SSIM_MRI'] = ssim(img2, fused_img, data_range=fused_img.max() - fused_img.min())
    metrics['MSE_CT'] = mean_squared_error(img1, fused_img)
    metrics['MSE_MRI'] = mean_squared_error(img2, fused_img)
    metrics['MI_CT'] = mutual_information(img1, fused_img)
    metrics['MI_MRI'] = mutual_information(img2, fused_img)
    # Add more metrics here if needed
    return metrics

# Load your images
img1 = cv2.imread('D:/SDP/Imfusion-main/screenshots/medical1.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('D:/SDP/Imfusion-main/screenshots/medical2.png', cv2.IMREAD_GRAYSCALE)
fused_img = cv2.imread('D:/SDP/Imfusion-main/Fused_Images/fused.png', cv2.IMREAD_GRAYSCALE)

# Calculate the metrics
results = calculate_metrics(img1, img2, fused_img)
print(results)


{'SSIM_CT': 0.3814543565102585, 'SSIM_MRI': 0.6364497066122207, 'MSE_CT': 1177.9404761904761, 'MSE_MRI': 1214.2030677655678, 'MI_CT': 1.4272309792791746, 'MI_MRI': 2.460728723690644}


1. Structural Similarity Index Measure (SSIM)
SSIM for CT (0.381): This is relatively low, indicating that the structural similarity between the fused image and the original CT scan is not very high. This suggests that some structural details from the CT image are not well preserved in the fused image.
SSIM for MRI (0.636): This is moderately high, showing that the fused image has a better preservation of structural details from the MRI scan compared to the CT scan. This suggests that the fusion process is more aligned with the characteristics of the MRI.


3. Mean Square Error (MSE)
MSE for CT (1177.94): A higher MSE value indicates a larger average squared difference between the pixel values of the fused image and the original CT image, which points to greater deviation and less accurate representation of the CT in the fusion.
MSE for MRI (1214.20): Similarly high, indicating significant differences between the fused image and the MRI image. This suggests that while structural similarities might be somewhat retained (as seen in SSIM), the exact pixel values differ considerably.


5. Mutual Information (MI)
MI for CT (1.427): This value shows moderate mutual information between the fused image and the CT scan, indicating that some amount of shared information exists, but it's not particularly high.
MI for MRI (2.461): This is higher compared to the CT, suggesting that the fused image shares more information with the MRI scan. This aligns with the SSIM results, further confirming that the MRI features are better represented in the fused image.


Conclusion


The metrics suggest that the fusion process tends to favor the MRI image over the CT scan in terms of preserving both structural and informational content. This could be due to inherent characteristics of the images, the specific settings of the fusion process, or the types of details (e.g., soft tissues in MRI vs. hard tissues in CT) that are better captured in MRI scans. The quality of the fusion, particularly with respect to the CT image, could be improved by adjusting the fusion algorithm to better balance the representation of both types of images.

For better fusion quality:

Consider using fusion techniques that can better handle the disparities in image characteristics between CT and MRI.
Adjust parameters or explore different fusion rules that might enhance the preservation of CT image details.
Experiment with preprocessing steps that might make the CT and MRI images more compatible for fusion, such as intensity or contrast adjustments.
This analysis can guide further optimizations and adjustments to the image fusion process to achieve a more balanced and effective representation of both CT and MRI characteristics in the fused image.

## No-Reference

In [7]:
import cv2
import numpy as np

def calculate_entropy(image):
    """Calculate the entropy of an image."""
    # Convert image to grayscale if it's not already
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate histogram from 0 to 255
    hist = cv2.calcHist([image], [0], None, [256], [0,256])
    # Normalize the histogram
    hist_norm = hist.ravel()/hist.sum()
    # Calculate the entropy
    logs = np.log2(hist_norm + 1e-10)  # Add a small value to avoid log(0)
    entropy = -1 * (hist_norm * logs).sum()
    
    return entropy

# Load your fused image
fused_img = cv2.imread('D:/SDP/Imfusion-main/Fused_Images/fused.png', cv2.IMREAD_GRAYSCALE)

# Calculate the entropy
entropy_value = calculate_entropy(fused_img)
print(f"Entropy: {entropy_value}")



Entropy: 5.897526264190674


Summary of the Fused Image's Entropy:

Value: An entropy of 5.90 indicates a moderate to high level of information content in the fused image. This suggests that the fusion process has managed to retain a good amount of information from the original CT and MRI images.

Interpretation: Higher entropy values generally mean that the image contains a wide variety of pixel values, implying more texture or detail. This can be viewed as a positive aspect of the fusion process, indicating that it has not resulted in excessive information loss.

Conclusion: Based on the entropy alone, the fusion effect can be considered effective to a certain degree. The image retains diverse information, which could be beneficial for further analysis or diagnostic purposes in medical imaging scenarios.

This metric provides a useful standalone assessment of the fused image's quality in terms of information retention. However, for a comprehensive evaluation, it would be ideal to consider this metric in conjunction with other reference and no-reference metrics that assess different aspects of image quality.

In [8]:
import cv2
import numpy as np

def calculate_average_gradient(image):
    """Calculate the average gradient of an image."""
    # Ensure the image is in grayscale
    if len(image.shape) != 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate gradients in the x and y directions
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    
    # Calculate the magnitude of the gradients
    grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    
    # Calculate the average gradient
    average_gradient = np.mean(grad_magnitude)
    
    return average_gradient

# Load your fused image
fused_img = cv2.imread('D:/SDP/Imfusion-main/Fused_Images/fused.png', cv2.IMREAD_GRAYSCALE)

# Calculate the average gradient
average_gradient_value = calculate_average_gradient(fused_img)
print(f"Average Gradient: {average_gradient_value}")


Average Gradient: 78.01037390623792


Summary of Average Gradient:

Value: An AG of 78.01 is generally considered high in the context of image processing, suggesting that the fused image retains a good amount of detail and texture from the original images.

Interpretation: Higher AG values are indicative of sharp edges and clear textures, which are essential for detailed visualization in applications such as medical imaging. It suggests that the image fusion process has effectively combined the relevant features from both the CT and MRI scans into the fused image without losing significant detail.

Conclusion: 

The fusion process appears to be successful in maintaining the structural integrity and detail from the source images. This is particularly important in medical imaging where the clarity of structural and textural details can be crucial for diagnosis and analysis.


Overall Impression:

Given the high Average Gradient value, the fusion technique used seems to effectively enhance or preserve the essential features from the original images, making the fused image potentially more useful for further analysis or clinical interpretation. This metric complements the entropy measure, providing a more comprehensive view of the fusion quality—where entropy assesses the information content, and average gradient evaluates the visual quality and sharpness.

Combining these insights, the fused image not only contains a high level of information (as indicated by entropy) but also presents this information with clarity and detail (as indicated by the average gradient). This makes for a robust assessment of the fusion's effectiveness.

In [10]:
import cv2
import numpy as np

def edge_similarity(image1, image2, fused_image):
    """Calculate the edge-based similarity measurement between source images and fused image."""
    # Compute gradients for source images and fused image
    grad_x1, grad_y1 = cv2.Sobel(image1, cv2.CV_64F, 1, 0), cv2.Sobel(image1, cv2.CV_64F, 0, 1)
    grad_x2, grad_y2 = cv2.Sobel(image2, cv2.CV_64F, 1, 0), cv2.Sobel(image2, cv2.CV_64F, 0, 1)
    grad_xf, grad_yf = cv2.Sobel(fused_image, cv2.CV_64F, 1, 0), cv2.Sobel(fused_image, cv2.CV_64F, 0, 1)
    
    # Calculate gradient magnitudes and angles
    magnitude1, angle1 = cv2.cartToPolar(grad_x1, grad_y1)
    magnitude2, angle2 = cv2.cartToPolar(grad_x2, grad_y2)
    magnitudef, anglef = cv2.cartToPolar(grad_xf, grad_yf)
    
    # Calculate edge information retention (intensity and orientation)
    qa_f = magnitude1 * (1 - np.abs(np.sin(angle1 - anglef)))
    qb_f = magnitude2 * (1 - np.abs(np.sin(angle2 - anglef)))
    
    # Define weights for each source image (example: equal weights)
    wa = wb = 0.5  # Adjust these weights as per the importance of each source image
    
    # Calculate edge-based similarity measure
    numerator = np.sum(qa_f * wa + qb_f * wb)
    denominator = np.sum(wa + wb)
    
    qab_f = numerator / denominator if denominator != 0 else 0
    
    return qab_f
    
def resize_images(*images):
    # Find the smallest dimensions in the set
    min_height = min(image.shape[0] for image in images)
    min_width = min(image.shape[1] for image in images)
    resized_images = [cv2.resize(img, (min_width, min_height), interpolation=cv2.INTER_AREA) for img in images]
    return resized_images

# Load your images
img1 = cv2.imread('D:/SDP/Imfusion-main/screenshots/medical1.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('D:/SDP/Imfusion-main/screenshots/medical2.png', cv2.IMREAD_GRAYSCALE)
fused_img = cv2.imread('D:/SDP/Imfusion-main/Fused_Images/fused.png', cv2.IMREAD_GRAYSCALE)


img1, img2, fused_img = resize_images(img1, img2, fused_img)

# Calculate the edge-based similarity measurement
qab_f_value = edge_similarity(img1, img2, fused_img)
print(f"Edge-based Similarity Measurement (QAB/F): {qab_f_value}")


Edge-based Similarity Measurement (QAB/F): 642967.07013141


The Edge-based Similarity Measurement (QAB/F) for your fused image is 642967.07, which is a notably high value. This indicates a strong retention of edge information from the original CT and MRI scans in the fused image. Here's a detailed interpretation and summary of this result:

Summary of Edge-based Similarity Measurement:

Value: A QAB/F value of approximately 642967 suggests that the edges from the source images are very well preserved in the fused image.

Interpretation: Edge information is critical in medical imaging as it helps in delineating structures and identifying features essential for accurate diagnosis. High edge retention indicates that the fusion process has successfully integrated the important features from both source images without significant degradation or loss of detail.

Conclusion: The fusion technique used is highly effective in terms of preserving structural integrity through edges, which is crucial for the visual clarity and utility of the image in clinical settings. This suggests that the fused image maintains a high level of detail, making it valuable for further analysis or interpretation in medical applications.


Overall Impression:

Given the high QAB/F value, the fusion process appears to be exceptionally successful in combining the CT and MRI images in a way that maintains their most critical visual information—edges. This metric, alongside the previously calculated entropy and average gradient, offers a comprehensive view of the fusion quality. High entropy and average gradient, coupled with excellent edge retention, denote a fusion outcome that is likely to be of high quality and utility.

These metrics collectively suggest that the fused image not only contains a significant amount of information but also presents it with clarity and precision, making it a potentially effective tool in medical imaging scenarios where detailed and accurate visual representations are necessary.

In [1]:
import cv2
import numpy as np

def calculate_standard_deviation(image):
    """Calculate the standard deviation of an image."""
    # Ensure the image is in grayscale
    if len(image.shape) != 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate mean
    mean_value = np.mean(image)
    # Calculate standard deviation
    standard_deviation = np.sqrt(np.mean((image - mean_value) ** 2))
    
    return standard_deviation

# Load your fused image
fused_img = cv2.imread('D:/SDP/Imfusion-main/Fused_Images/fused.png', cv2.IMREAD_GRAYSCALE)

# Calculate the standard deviation
standard_deviation_value = calculate_standard_deviation(fused_img)
print(f"Standard Deviation: {standard_deviation_value}")


Standard Deviation: 46.81556904792552


Summary of Standard Deviation:

Value: An SD of 46.82 is relatively high, indicating a significant level of contrast within the fused image.

Interpretation: High standard deviation in an image implies that there is a wide range of pixel intensities. This diversity in pixel values generally enhances the image's contrast, making the details within the image more distinguishable and the overall image more dynamic and visually informative.

Conclusion: 

The higher contrast denoted by the substantial SD value suggests that the fusion process effectively enhances or maintains the dynamic range of the original images. This is desirable in many applications, particularly in medical imaging, where contrast plays a crucial role in the visibility of various anatomical structures and abnormalities.


Overall Impression:

The higher Standard Deviation value indicates that the fusion process used is successful in retaining or even enhancing the contrast and dynamic range from the original CT and MRI scans. This contrast is crucial for ensuring that important features within the images are pronounced and easily discernible, which can be vital for subsequent analysis, diagnosis, or any automated processing tasks.

In [2]:
import cv2
import numpy as np

def calculate_spatial_frequency(image):
    """Calculate the spatial frequency of an image."""
    # Ensure the image is in grayscale
    if len(image.shape) != 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate differences between adjacent pixels along rows and columns
    row_diff = np.diff(image.astype('float'), axis=0)
    col_diff = np.diff(image.astype('float'), axis=1)
    
    # Calculate the squares of differences
    RF_squared = np.sum(row_diff ** 2)
    CF_squared = np.sum(col_diff ** 2)
    
    # Calculate row frequency (RF) and column frequency (CF)
    RF = np.sqrt(RF_squared / (image.shape[0] * image.shape[1]))
    CF = np.sqrt(CF_squared / (image.shape[0] * image.shape[1]))
    
    # Calculate spatial frequency (SF)
    SF = np.sqrt(RF ** 2 + CF ** 2)
    
    return SF

# Load your fused image
fused_img = cv2.imread('D:/SDP/Imfusion-main/Fused_Images/fused.png', cv2.IMREAD_GRAYSCALE)

# Calculate the spatial frequency
spatial_frequency_value = calculate_spatial_frequency(fused_img)
print(f"Spatial Frequency: {spatial_frequency_value}")


Spatial Frequency: 28.85894511880092


Summary of Spatial Frequency:

Value: An SF of 28.86 suggests that there is a significant amount of texture and edge detail in the fused image. This is a relatively high value that points towards an active variation in pixel intensity across the image.

Interpretation: Spatial Frequency measures the variations in pixel intensities which correspond to edges and textural details. A higher SF value generally indicates that these details are prevalent and well-preserved or even enhanced in the fused image. This is beneficial for visual clarity and the usefulness of the image in applications where detail perception is crucial.

Conclusion: 

The fusion process appears to be effective in maintaining or enhancing the textural and edge details from the original CT and MRI scans. This suggests that the fused image not only retains important information from both source images but also presents this information in a manner that is visually rich and detailed.


Overall Impression:


Given the calculated metrics of entropy, average gradient, edge-based similarity, standard deviation, and now spatial frequency, your fused image maintains a high level of information content, edge detail, and textural complexity. These characteristics are indicative of a high-quality image fusion process that enhances the utility of the resulting image for detailed analysis or diagnostic purposes. The higher spatial frequency specifically confirms that the image is likely to be more effective in applications that rely on detailed visual inspections, such as medical imaging.

In [3]:
import cv2
import numpy as np

def calculate_edge_intensity(image):
    """Calculate the edge intensity of an image using Sobel operators."""
    # Ensure the image is in grayscale
    if len(image.shape) != 2:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Sobel operators to find horizontal and vertical gradients
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    
    # Calculate the squares of gradients
    S2x = np.sum(sobel_x**2)
    S2y = np.sum(sobel_y**2)
    
    # Calculate Edge Intensity
    EI = np.sqrt(S2x + S2y)
    
    return EI

# Load your fused image
fused_img = cv2.imread('D:/SDP/Imfusion-main/Fused_Images/fused.png', cv2.IMREAD_GRAYSCALE)

# Calculate the edge intensity
edge_intensity_value = calculate_edge_intensity(fused_img)
print(f"Edge Intensity: {edge_intensity_value}")


Edge Intensity: 14363.941381111244


Summary of Edge Intensity:

Value: An EI of approximately 14363.94 is significantly high, reflecting a strong presence of edge details in the fused image.

Interpretation: Edge intensity is a crucial metric in evaluating the quality of image fusion, especially for its ability to highlight how well the fusion process preserves or enhances the edges from the source images. Edges are essential for delineating boundaries and features within the image, and a higher edge intensity indicates that these features are both clear and prominent.

Conclusion: 

The high edge intensity value suggests that the fusion technique effectively consolidates and even amplifies edge information from the CT and MRI scans. This ensures that the fused image maintains high clarity, making it more useful for applications that rely heavily on edge detail for analysis, such as medical imaging diagnostics or feature-based geospatial analysis.


Overall Impression:

Given the calculated edge intensity, along with the previous metrics like entropy, average gradient, edge-based similarity measurement, standard deviation, and spatial frequency, it is evident that the fused image is of high quality. The consistency in high metric values across different measures—reflecting detail retention, information content, and visual clarity—indicates a successful image fusion process.